# Script for training with decoupled shape parameters

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from skimage.io import imread
from tqdm import tqdm
import torch
import numpy as np
import mediapipe as mp
import os
import cv2
import trimesh
import matplotlib.pyplot as plt
import logging
import ipywidgets as widgets
from IPython.display import display

Function to get the Shape parameters for an image -> we need this later

In [ ]:
def get_shape_params(image_path: str, device='cuda'):
    """Function for getting the shape paramters of an image
    """
    # Initialization
    deca = DECA()

    #Get the data in the right format
    test_img = datasets.TestData(image_path)
    deca_cfg.model.use_tex = False
    deca_cfg.model.extract_tex = False
    #Get the codedictionarys
    for i in tqdm(range(len(test_img))):
        name = test_img[i]['imagename']
        images_l = test_img[i]['image'].to(device)[None,...]
        with torch.no_grad(): 
            codedict = deca.encode(images_l)
            
    return codedict['shape']

For every image, we need to get the shape parameters 

In [ ]:
print(get_shape_params('/home/dietrich/Testing/DECA/DECA/TestSamples/Brenken_Rudolf/2017-10-24_00-00_Brenken-Rudolf_06/2017-10-24_001.jpg'))

Get the landmarks

In [ ]:
images_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/Images/VGGface2_None_norm_512_true_bygfpgan'
lmks_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/lmks_train'
shape_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/shape_train'
img_name_list = []

total_steps = 9279
counter = 0

progress_slider = widgets.IntSlider(min=0, max=total_steps, value=counter, description='Progress')
logging.basicConfig(filename='/home/dietrich/Testing/DECA/Dataset/VGGFACE/lmks_and_shape_processing_6_imgs.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def update_slider(value):
    progress_slider.value = value
display(progress_slider)

lms_478_to_68 = [
                162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87
            ]

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
                static_image_mode = True,
                max_num_faces = 1,
                refine_landmarks=False,
                )

expected_landmarks_count = 34 # Half should be detected
# Through all folders in the Dataset Path
for individ__name in os.listdir(images_folder):
    individ__path = os.path.join(images_folder, individ__name)

    if os.path.isdir(individ__path):
        lmks_individ_path = os.path.join(lmks_folder, individ__name)
        if not os.path.exists(lmks_individ_path):
            os.makedirs(lmks_individ_path)
        # e.g. K - number of images per person
        img_num = 6
        # counting valid images and break, if enough images are taken
        img_list_individual = []
        # For one individual (one folder) going through the images, until 6 are detected
        for image_name in os.listdir(individ__path):
            image_path = os.path.join(individ__path, image_name)

            if image_name.endswith('.jpg'):
                fn, ext = os.path.splitext(os.path.basename(image_name))

                #Process Mediapipe
                image = cv2.imread(image_path)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('uint8')

                results = face_mesh.process(image_rgb)
                
                # Excactly one person should be detected, else the image is not used
                if results.multi_face_landmarks and len(results.multi_face_landmarks) == 1:
                    landmarks = results.multi_face_landmarks[0].landmark
                    selected_lm = [landmarks[i] for i in lms_478_to_68] 
                    selected_lm = [[lm.x * image.shape[1], lm.y * image.shape[0]] for lm in selected_lm]

                    # Half of the face need to be detected, else another image is used
                    if len(selected_lm) >= expected_landmarks_count:
                        np.save(os.path.join(lmks_individ_path, fn + "_68kpts.npy") ,selected_lm)
                        img_list_individual.append(os.path.join(individ__name, fn))
                    else:
                        logging.info(f'Not used: Detected less than 35 Landmarks: {individ__path}/{image_name}')
                else:
                    logging.info(f'Not used: Detected none or more than one face: {individ__path}/{image_name}')

            #If we got 6 Images, the next individual will be landmarked
            if len(img_list_individual) == img_num:
                img_name_list.append(img_list_individual)
                np.save("/home/dietrich/Testing/DECA/Dataset/VGGFACE/"+"data_names_6_per_individual", img_name_list)
                break
    counter +=1
    update_slider(counter)
    
print("Landmark Detection and Image Selection Done")



Test, if getting shape parameters is working the right way

In [ ]:
datafile = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual.npy'
data_lines = np.load(datafile).astype('str')

In [ ]:
shape_list = []
base_path = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/Images/VGGface2_None_norm_512_true_bygfpgan'

idx = 0
for i in range(6):
    name = data_lines[idx, i]
    image_path = os.path.join(base_path, name + '.jpg') 
    shape_params = get_shape_params(image_path)
    shape_list.append(shape_params)

shape_2d = torch.stack(shape_list)
mean_shape = torch.mean(shape_2d, dim=0).cpu().detach().numpy()
mean_shape_list = [mean_shape for _ in shape_list]


shapearray = torch.from_numpy(np.array(mean_shape_list)).type(dtype = torch.float32) #K,100

print(shapearray)

Actual Training

In [ ]:
from decalib.deca import DECA
from decalib.trainer import Trainer
from decalib.utils.config_train import config_train as cfg
import yaml
import shutil
import torch.backends.cudnn as cudnn

In [ ]:
os.makedirs(os.path.join(cfg.output_dir, cfg.train.log_dir), exist_ok=True)
os.makedirs(os.path.join(cfg.output_dir, cfg.train.vis_dir), exist_ok=True)
os.makedirs(os.path.join(cfg.output_dir, cfg.train.val_vis_dir), exist_ok=True)
with open(os.path.join(cfg.output_dir, cfg.train.log_dir, 'full_config.yaml'), 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)
shutil.copy(cfg.cfg_file, os.path.join(cfg.output_dir, 'config.yaml'))
    
# cudnn related setting
cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.enabled = True

# start training
# deca model
cfg.rasterizer_type = 'pytorch3d'
deca = DECA(cfg)
trainer = Trainer(model=deca, config=cfg)

## start train
trainer.fit()